In [1]:
from concurrent.futures import ThreadPoolExecutor, as_completed

# created libraries for the project
from web_scraper import get_product_ids
# from utility import create_product_directory
from multi_thread_module import process_product
import global_vars


In [2]:
class AC_motor:

    def __init__(self,type, sub_type, sub_sub_type):
        self.type = type
        self.sub_type = sub_type
        self.sub_sub_type = sub_sub_type
        self.product_web_page_code = list(global_vars.dict_ac_motors_general_purpose[self.sub_sub_type].keys())[0]
        self.product_specs = list(global_vars.dict_ac_motors_general_purpose[self.sub_sub_type][self.product_web_page_code])


In [3]:
MAX_THREADS = 10


if __name__ == '__main__':

    list_product_web_page_code = [110, 312]
    list_sub_sub_type = ["Three Phase Enclosed","Single Phase Enclosed"]
    # list_product_web_page_code = [110]
    # list_sub_sub_type = ["Three Phase Enclosed"]

    for sub_sub_type in list_sub_sub_type:
        product_spec = []
        product_web_page_code = list(global_vars.dict_ac_motors_general_purpose[sub_sub_type].keys())[0]
        print(product_web_page_code)
        
        product_spec = global_vars.dict_ac_motors_general_purpose[sub_sub_type][product_web_page_code]
        print(product_spec)

        list_product_codes, list_image_id = get_product_ids(product_web_page_code)
        

        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = []
            # for product_code in list_product_codes:
            for product_code,image_id in zip(list_product_codes, list_image_id):
                futures.append(executor.submit(process_product, product_code, image_id, product_spec))

            for future in as_completed(futures):
                future.result()  # Captura possíveis exceções


110
['Catalog Number', 'Enclosure', 'Frame', 'Frame Material', 'Frequency', 'Motor Letter Type', 'Output @ Frequency', 'Phase', 'Synchronous Speed @ Frequency', 'Voltage @ Frequency', 'Haz Area Class and Group', 'Haz Area Division', 'Agency Approvals', 'Ambient Temperature', 'Auxillary Box', 'Auxillary Box Lead Termination', 'Base Indicator', 'Bearing Grease Type', 'Blower', 'Current @ Voltage', 'Design Code', 'Drip Cover', 'Duty Rating']
https://www.baldor.com/api/products?include=results&language=en-US&include=filters&include=category&pageSize=10&category=110
erro pegar img id
erro pegar img id
erro pegar img id
erro pegar img id
erro pegar img id
erro pegar img id
erro pegar img id
erro pegar img id
Page 0 loaded successfully.
{'name': '3-Phase CEBM3546T ', 'Quantity': '1', 'product_name': 'CEBM3546T', 'description': '1HP, 1770RPM, 3PH, 60HZ, 143TC, 3522M, TEFC, F1', 'Catalog Number': 'CEBM3546T', 'Enclosure': 'TEFC', 'Frame': '143TC', 'Frame Material': 'Steel', 'Frequency': '60.00 